In [1]:
# each of these keys are columns in final DF
prop_type_d = {
    'size': {"Small", "Tiny", "Bulky"},
    'chain type': {'Aliphatic', 'Aromatic'},
    'polar': {'Non-polar', 'Polar', 'Charged'},
    'hydrophobic': {'Hydrophobic'},
    'iron binding': {'Iron binding'},
    'order': {'Order promoting', 'Disorder promoting', 'Disorder promoting if in secstruct'},
    'carbonylation': {'Carbonylation susceptible'},
    'oxygen rich': {'Oxygen rich'},
    'photooxidation': {'Prone to photooxidation'},
    'pathogen': {'Pathogen enriched'},
    'TM stabilizing': {'TM stabilizing', 'TM to Thr stabilizing'},
    'common in': {"common in helix", "common in turn", "common in sheet"}
}

In [2]:
# Having to do this since there's a bug in ssbio

_aa_property_dict_one = {
    'Aliphatic': {'A', 'I', 'L', 'V'},
    'Aromatic' : {'F', 'H', 'W', 'Y'},
    'Non-polar': {'A', 'C', 'F', 'G', 'I', 'L', 'M', 'P', 'V', 'W', 'Y'},
'Polar'    : {'D', 'E', 'H', 'K', 'N', 'Q', 'R', 'S', 'T'},
'Charged'  : {'D', 'E', 'H', 'K', 'R'},
    'Basic'    : {'H', 'K', 'R'},
    'Acidic'   : {'D', 'E'},
#     'Small'    : {'A', 'C', 'D', 'G', 'N', 'P', 'S', 'T', 'V'}}  # I'm going to consider V in bulky according to paper that calls anything bigger than 18 A^2 as bulky https://www.sciencedirect.com/science/article/pii/0022519368900696
    'Small'    : {'A', 'C', 'D', 'G', 'N', 'P', 'S', 'T'},
    'Tiny'     : {'A', 'C', 'G', 'S', 'T'},
}

EXTENDED_AA_PROPERTY_DICT_ONE = {}
EXTENDED_AA_PROPERTY_DICT_ONE['Bulky'] = {'L', 'I', 'F', 'Y', 'V', 'W'}  # https://www.sciencedirect.com/science/article/pii/0022519368900696
EXTENDED_AA_PROPERTY_DICT_ONE['Hydrophobic'] = {'L', 'M', 'F', 'W', 'C', 'A', 'V', 'G', 'I', 'Y', 'H', 'T', 'S','P'}  # http://www.russelllab.org/aas/hydrophobic.html
EXTENDED_AA_PROPERTY_DICT_ONE['Iron binding'] = {'D', 'E', 'Y', 'H', 'C'}  # http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0007269
EXTENDED_AA_PROPERTY_DICT_ONE['Disorder promoting'] = {'P', 'E', 'S', 'K', 'Q', 'G'}  # https://www.sciencedirect.com/science/article/pii/S1570963912002889
EXTENDED_AA_PROPERTY_DICT_ONE['Disorder promoting if in secstruct'] = {'G', 'P'}  # https://www.sciencedirect.com/science/article/pii/S2211124714003751
EXTENDED_AA_PROPERTY_DICT_ONE['Order promoting'] = {'W', 'F', 'Y', 'I','M'}  # https://www.sciencedirect.com/science/article/pii/S2211124714003751 https://www.sciencedirect.com/science/article/pii/S1570963912002889#bb0185
EXTENDED_AA_PROPERTY_DICT_ONE['Carbonylation susceptible'] = {'P', 'R', 'K', 'T'}  # http://onlinelibrary.wiley.com/doi/10.1002/pmic.201100223/full
EXTENDED_AA_PROPERTY_DICT_ONE['Oxygen rich'] = {'D', 'E', 'N', 'Q', 'S', 'T', 'Y'}  # http://www.sciencedirect.com/science/article/pii/S2211124714003751
EXTENDED_AA_PROPERTY_DICT_ONE['Prone to photooxidation'] = {'C', 'H', 'W', 'M','Y'}  # listed in decreasing order, https://www.sciencedirect.com/science/article/pii/S1011134401002081
EXTENDED_AA_PROPERTY_DICT_ONE['Pathogen enriched'] = {'G', 'P', 'R', 'S'}  # https://www.sciencedirect.com/science/article/pii/S2211124714003751
EXTENDED_AA_PROPERTY_DICT_ONE['TM stabilizing'] = {'T', 'C', 'S'}  # http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0003343
EXTENDED_AA_PROPERTY_DICT_ONE['TM to Thr stabilizing'] = {'I', 'M', 'V', 'A'}  # http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0003343

# I've added these from the _human_readable_pepstats dictionary within ssbio/protein/sequence/properties/residues.py
EXTENDED_AA_PROPERTY_DICT_ONE["common in helix"] = {'V', 'I', 'Y', 'F', 'W', 'L'}
EXTENDED_AA_PROPERTY_DICT_ONE["common in turn"] = {'N', 'P', 'G', 'S'}
EXTENDED_AA_PROPERTY_DICT_ONE["common in sheet"] = {'E', 'M', 'A', 'L'}


from copy import deepcopy

def characterize_residue_mutation(res1, res2, use_extended_def=True):
    res1_props = []
    res2_props = []

    propdict = deepcopy(_aa_property_dict_one)

    if use_extended_def:
        propdict.update(EXTENDED_AA_PROPERTY_DICT_ONE)

    for prop, aa_list in propdict.items():  # bug is here; EXTENDED_AA_PROPERTY_DICT_ONE is used instead of propdict
        if res1 in aa_list:
            res1_props.append(prop)
        if res2 in aa_list:
            res2_props.append(prop)

    return {res1: res1_props,
            res2: res2_props}

In [3]:
import ssbio.protein.sequence.properties.residues as residue_props

In [4]:
import pandas as pd
meta_muts = pd.read_pickle("./glpK_muts.pkl")  # TODO: rename to glpK_muts
meta_muts.head()

FileNotFoundError: [Errno 2] No such file or directory: './glpK_muts.pkl'

In [ ]:
aa_hydro_d = residue_props.kd_hydrophobicity_one
aa_bulki_d = residue_props.bulkiness_one

# This will make a table to inspect all of the changes for all of the mutations.

aa_flex_d = residue_props._aa_flexibility_dict_one


snps = meta_muts[meta_muts["Mutation Type"]=="SNP"]
SNP_AA_changes = snps[["AA change"]]
unique_SNP_AA_changes = SNP_AA_changes.drop_duplicates()  # Focused on AA positions and not mutation frequencies

df = pd.DataFrame()
for _, r in unique_SNP_AA_changes.iterrows():
    pos = int(r["AA change"][1:][:-1])
#     sub_genic_feats = _get_mut_subgenic_feats(r["AA range"], gff_df)  # TODO: if not on feature, populate this with "nearest" feature in 3D space.
    aa1 = r["AA change"][0]
    aa2 = r["AA change"][-1]
    props = characterize_residue_mutation(aa1, aa2, use_extended_def=True)
    d = {"position": pos, "AA": (aa1, aa2)}
    for k, v in prop_type_d.items():
        k1 = k + ' 1'
        ps = v & set(props[aa1])
        if ("Small" in ps) & ("Tiny" in ps):
            ps.remove("Small")
        d[k1] = ', '.join(ps)
        
        k2 = k + ' 2'
        ps = v & set(props[aa2])
        if ("Small" in ps) & ("Tiny" in ps):
            ps.remove("Small")
        d[k2] = ', '.join(ps)
        
        d["size diff"] = aa_bulki_d[aa2] - aa_bulki_d[aa1]
        d["hydrophob diff"] = aa_hydro_d[aa2] - aa_hydro_d[aa1]
        d["flex diff"] = aa_flex_d[aa2] - aa_flex_d[aa1]
        
    df = df.append(d, ignore_index=True)
    
df["position"] = df["position"].astype(int)
df = df.sort_values(by=["position", "AA"])
df = df.set_index("position")
df = df.sort_index(axis=1)
df


# This outputs the table in a smaller format for presentation
tdf = df[
    ((df.index>=50) & (df.index<=68))
#     | ((df.index>=72) & (df.index<=74))
#     | (df.index==173)  # Don't have previously existing functional analysis for this
  ]
# tdf[["AA", "chain type 1", "chain type 2", "size 1", "size 2", "size diff", "polar 1", "polar 2"]]
tdf

In [ ]:
aa_set = {'R', 'H', 'K', 'D', 'E', 'S', 'T', 'N', 'Q', 'C', 'G', 'P', 'A', 'I', 'L', 'M', 'F', 'W', 'Y', 'V'}
from copy import deepcopy

def characterize_residue(res):
    res_props = []

    propdict = deepcopy(_aa_property_dict_one)
    propdict.update(EXTENDED_AA_PROPERTY_DICT_ONE)

    for prop, aa_list in propdict.items():
        if res in aa_list:
            res_props.append(prop)

    return {res: res_props}


aa_pos_prop_df = pd.DataFrame(columns=['AA']+list(prop_type_d.keys()))
for aa in aa_set:
    props = characterize_residue(aa)
    d = {"AA": aa}
    
    case = ''
    if aa == 'S':
        case = "WT"
    if aa == 'W':
        case = "Bystrom99"
    if aa == 'Y':
        case = "ALE"
    d["case"] = case
    
    for k, v in prop_type_d.items():
        ps = v & set(props[aa])
        if ("Small" in ps) & ("Tiny" in ps):
            ps.remove("Small")
        d[k] = ', '.join(ps)
        d["size val"] = aa_bulki_d[aa]
        d["hydrophobicity"] = aa_hydro_d[aa]
        d["flexibility"] = aa_flex_d[aa]
        
    aa_pos_prop_df = aa_pos_prop_df.append(d, ignore_index=True)
aa_pos_prop_df = aa_pos_prop_df.sort_values(by=["case", "size"], ascending=False)
aa_pos_prop_df = aa_pos_prop_df.set_index('AA', drop=True)
aa_pos_prop_df = aa_pos_prop_df[['case', 'size', 'chain type', 'polar', 'hydrophobic',
                                 'iron binding', 'order', 'carbonylation', 'oxygen rich',
                                 'photooxidation', 'pathogen', 'TM stabilizing', 'common in',
                                 'hydrophobicity', 'size val', "flexibility"]]
# df = df.reindex(['S', 'V', 'Y', 'N', 'D', 'E', 'C', 'F', 'I', 'P', 'R', 'M', 'G', 'H', 'A', 'L', 'K', 'T', 'Q', 'W'])
aa_pos_prop_df

In [ ]:
all_glpK_sift_df = pd.read_csv('../data/mutfunc_full/glpK_sift.tab', sep='\t', header=None)
all_glpK_sift_df.columns = ["acc","pos","ref","alt","score","median_ic","n_aa","n_seq"]
# Don't have to add 1 to position for this file,
# which makes sense in that mutfunc's webapp called the correct AAs with it's SIFT results.
aa59_glpK_sift_df = all_glpK_sift_df[all_glpK_sift_df["pos"]==59].copy()
aa59_glpK_sift_df = aa59_glpK_sift_df.set_index('alt', drop=True)
aa59_glpK_sift_df.head()

In [ ]:
aa_pos_prop_df["SIFT (*≤0.05)"] = aa_pos_prop_df.apply(lambda r: aa59_glpK_sift_df.loc[r.name, "score"], axis=1)
aa_pos_prop_df

In [ ]:
# getting interface regions from mutfunc's precalculated outputs
# Only 2 structures are currently being included with these interface regions (3EZW, 1GLB) and both of the AA sequences align with that in NC_000913.3
# Structure 3EZW best represents BOP27 GlpK of the two structures available since 1GLB already has a mutation to residue 65 (tetramer disruption)
# therefore can use the AA positions freely between all.

all_glpK_interface_df = pd.read_csv('../data/mutfunc_full/glpK_interfaces.tab', sep='\t', header=None)
all_glpK_interface_df.columns = ["pdb","prot_a","prot_b","evidence","chain","aa_wt","aa_mt","pdb_pos","uniprot_pos","mut","dG_wt","dG_mt","ddG","dG_wt_sd","dG_mt_sd", "ddG_sd"]
all_glpK_interface_df["NC_000913v3_pos"] = all_glpK_interface_df["uniprot_pos"] + 1  # Adding 1 to account for initiator methionine (start codon) within NC_000913v3 AA ref
aa59_glpK_interface_df = all_glpK_interface_df[all_glpK_interface_df["NC_000913v3_pos"]==59]
aa59_glpK_interface_df.head()

In [ ]:
# In the case of glpK and uniprot_pos, they do align
all_glpK_exp_df = pd.read_csv('../data/mutfunc_full/glpK_exp.tab', sep='\t', header=None)
all_glpK_exp_df.columns=["pdb_id","chain","aa_wt","aa_mt","pdb_pos","uniprot_pos","uniprot_id","dG_wt","dG_mt","ddG","dG_wt_sd","dG_mt_sd","ddG_sd"]
aa59_glpK_exp_df = all_glpK_exp_df[all_glpK_exp_df["uniprot_pos"]==59]
aa59_glpK_exp_df

In [ ]:
def _get_ddG(aa_mut, aa_pos):
    return aa59_glpK_exp_df[((aa59_glpK_exp_df["uniprot_pos"]==aa_pos) & (aa59_glpK_exp_df["aa_mt"]==aa_mut))]["ddG"].mean()  # So that it matches the webapp calls.
#     return aa59_glpK_interface_df[((aa59_glpK_interface_df["NC_000913v3_pos"]==aa_pos) & (aa59_glpK_interface_df["aa_mt"]==aa_mut))]["ddG"].mean()


aa_pos_prop_df["ΔΔG (*>2)"] = aa_pos_prop_df.apply(lambda r: _get_ddG(r.name, 59), axis=1)
aa_pos_prop_df

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.sans-serif'] = ["Helvetica"]
sns.set_context("paper")
sns.set_style("ticks")

df = aa_pos_prop_df[["SIFT (*≤0.05)"]].copy()
df = df[~(df.index == 'S')]
df = df.sort_values(by="SIFT (*≤0.05)", ascending=False)
# display(df)


c_l = ['#587E8B'] * 19
c_l[-2] = '#CF9542'
c_l[-1] = "#A63F41"
c_l[-6] = "#79B552"
c_l = [c_l]
print(c_l)
ax = df.plot.barh(
    #     color="#4169e1",
    color=c_l,
    figsize=(1, 3),
    width=1
)

sns.despine(ax=ax, top=True, right=True, bottom=True, left=True)
ax.get_legend().remove()
ax.tick_params(axis='y', which='major', length=0)
# ax.get_xaxis().set_visible(False)

for tick in ax.get_xticklabels():
    tick.set_fontname("Helvetica")
for tick in ax.get_yticklabels():
    tick.set_fontname("Helvetica")

df = df.reset_index()  # Resetting index for the below annotations to work
for i, r in df.iterrows():
    s = str("%.3f" % r["SIFT (*≤0.05)"])
    c = "#000000"
    if r["AA"] == "Y":
        c = "#CF9542"
    elif r["AA"] == "W":
        c = "#A63F41"
    elif r["AA"] == "F":
        c = "#79B552"
    ax.text(r["SIFT (*≤0.05)"], i-0.4,  s, fontname="Helvetica", color=c)

ax.tick_params(axis='both', which='both', length=0)

for ytick in ax.get_yticklabels():
    if ytick.get_text() == 'Y':
        ytick.set_color("#CF9542")
    if ytick.get_text() == 'W':
        ytick.set_color("#A63F41")
    if ytick.get_text() == 'F':
        ytick.set_color("#79B552")

ax.set_xticklabels([])
ax.set_xlabel('SIFT (*≤0.05)', fontname="Helvetica")
ax.xaxis.labelpad = -2
ax.set_ylabel('residue', fontname="Helvetica")
ax.set_title("Deleterious\naccording to\nconservation", fontname="Helvetica")

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.sans-serif'] = ["Helvetica"]
sns.set_context("paper")
sns.set_style("ticks")

df = aa_pos_prop_df[["ΔΔG (*>2)"]].copy()
df = df[~(df.index=='S')]
df = df.sort_values(by="ΔΔG (*>2)", ascending=True)
df

c_l = ['#587E8B'] * 19
c_l[-11] = '#CF9542'
c_l[-13] = "#A63F41"
c_l[-15] = "#79B552"
c_l = [c_l]
ax = df.plot.barh(
    color=c_l,
    figsize=(1,3),
    width=1
)

sns.despine(ax=ax, top=True, right=True, bottom=True, left=True)
ax.get_legend().remove()
# plt.xscale('log')
ax.tick_params(axis='y', which='major', length=0)
# ax.get_xaxis().set_visible(False)

for tick in ax.get_xticklabels():
    tick.set_fontname("Helvetica")
for tick in ax.get_yticklabels():
    tick.set_fontname("Helvetica")

df = df.reset_index()  # Resetting index for the below annotations to work
for i, r in df.iterrows():
    c = "#000000"
    if r["AA"] == "Y":
        c = "#CF9542"
    elif r["AA"] == "W":
        c = "#A63F41"
    elif r["AA"] == "F":
        c = "#79B552"
    s = str("%.3f" % r["ΔΔG (*>2)"])
    if r["ΔΔG (*>2)"] < 0:
        ax.text(0, i-0.4,  s, fontname="Helvetica", color=c)
    else:
        ax.text(r["ΔΔG (*>2)"], i-0.4,  s, fontname="Helvetica", color=c)


ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_xlabel('ΔΔG (*>2)', fontname="Helvetica")
ax.set_ylabel('', fontname="Helvetica")
ax.xaxis.labelpad = -2

for ytick in ax.get_yticklabels():
    if ytick.get_text() == 'Y':
        ytick.set_color("#CF9542")
    if ytick.get_text() == 'W':
        ytick.set_color("#A63F41")
    if ytick.get_text() == 'F':
        ytick.set_color("#79B552")

ax.set_title("Structure\ndestabilizing", fontname="Helvetica")

In [ ]:
wt = 'S'
aa_pos_prop_df["size diff"] = aa_pos_prop_df.apply(lambda r: r["size val"] - aa_pos_prop_df.at[wt, "size val"], axis=1)
aa_pos_prop_df

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.sans-serif'] = ["Helvetica"]
sns.set_context("paper")
sns.set_style("ticks")

df = aa_pos_prop_df[["size diff"]].copy()
df = df[~(df.index=='S')]
df = df.sort_values(by="size diff", ascending=True)
df

c_l = ['#587E8B'] * 19
c_l[-6] = '#CF9542'
c_l[-1] = "#A63F41"
c_l[-5] = "#79B552"
c_l = [c_l]
ax = df.plot.barh(
    color=c_l,
    figsize=(1,3),
    width=1
)

sns.despine(ax=ax, top=True, right=True, bottom=True, left=True)
ax.get_legend().remove()
# plt.xscale('log')
ax.tick_params(axis='y', which='major', length=0)
# ax.get_xaxis().set_visible(False)

for tick in ax.get_xticklabels():
    tick.set_fontname("Helvetica")
for tick in ax.get_yticklabels():
    tick.set_fontname("Helvetica")

df = df.reset_index()  # Resetting index for the below annotations to work
for i, r in df.iterrows():
    c = "#000000"
    if r["AA"] == "Y":
        c = "#CF9542"
    elif r["AA"] == "W":
        c = "#A63F41"
    elif r["AA"] == "F":
        c = "#79B552"
    s = str("%.1f" % r["size diff"])
    if r["size diff"] < 0:
        ax.text(0, i-0.4,  s, fontname="Helvetica", color=c)
    else:
        ax.text(r["size diff"], i-0.4,  s, fontname="Helvetica", color=c)
        
for ytick in ax.get_yticklabels():
    if ytick.get_text() == 'Y':
        ytick.set_color("#CF9542")
    if ytick.get_text() == 'W':
        ytick.set_color("#A63F41")
    if ytick.get_text() == 'F':
        ytick.set_color("#79B552")

ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_xlabel("Å^2", fontname="Helvetica")
ax.set_ylabel("", fontname="Helvetica")
ax.xaxis.labelpad = -2

ax.set_title("Size difference", fontname="Helvetica")

In [ ]:
wt = 'S'
aa_pos_prop_df["flex diff"] = aa_pos_prop_df.apply(lambda r: r["flexibility"] - aa_pos_prop_df.at[wt, "flexibility"], axis=1)
aa_pos_prop_df

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.sans-serif'] = ["Helvetica"]
sns.set_context("paper")
sns.set_style("ticks")

df = aa_pos_prop_df[["flex diff"]].copy()
df = df[~(df.index=='S')]
df = df.sort_values(by="flex diff", ascending=False)
df

c_l = ['#587E8B'] * 19
c_l[-2] = '#CF9542'
c_l[-1] = "#A63F41"
c_l[-3] = "#79B552"
c_l = [c_l]
ax = df.plot.barh(
    color=c_l,
    figsize=(1,3),
    width=1
)

sns.despine(ax=ax, top=True, right=True, bottom=True, left=True)
ax.get_legend().remove()
# plt.xscale('log')
ax.tick_params(axis='y', which='major', length=0)
# ax.get_xaxis().set_visible(False)

for tick in ax.get_xticklabels():
    tick.set_fontname("Helvetica")
for tick in ax.get_yticklabels():
    tick.set_fontname("Helvetica")

df = df.reset_index()  # Resetting index for the below annotations to work
for i, r in df.iterrows():
    c = "#000000"
    if r["AA"] == "Y":
        c = "#CF9542"
    elif r["AA"] == "W":
        c = "#A63F41"
    elif r["AA"] == "F":
        c = "#79B552"
    s = str("%.3f" % r["flex diff"])
    if r["flex diff"] < 0:
        ax.text(0, i-0.4,  s, fontname="Helvetica", color=c)
    else:
        ax.text(r["flex diff"], i-0.4,  s, fontname="Helvetica", color=c)
        
for ytick in ax.get_yticklabels():
    if ytick.get_text() == 'Y':
        ytick.set_color("#CF9542")
    if ytick.get_text() == 'W':
        ytick.set_color("#A63F41")
    if ytick.get_text() == 'F':
        ytick.set_color("#79B552")

ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_xlabel("", fontname="Helvetica")
ax.set_ylabel("", fontname="Helvetica")
ax.xaxis.labelpad = -2

ax.set_title("Flexibility difference", fontname="Helvetica")

In [ ]:
wt = 'S'
aa_pos_prop_df["hydrophob diff"] = aa_pos_prop_df.apply(lambda r: r["hydrophobicity"] - aa_pos_prop_df.at[wt, "hydrophobicity"], axis=1)
aa_pos_prop_df

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.sans-serif'] = ["Helvetica"]
sns.set_context("paper")
sns.set_style("ticks")

df = aa_pos_prop_df[["hydrophob diff"]].copy()
df = df[~(df.index=='S')]
df = df.sort_values(by="hydrophob diff", ascending=True)
df

# df = df[-10:]
ax = df.plot.barh(
#     y="count",
#     x="index",
#     color="#4C72B0",
    color="#4169e1",
    figsize=(1,3),
    width=1
)

sns.despine(ax=ax, top=True, right=True, bottom=True, left=True)
ax.get_legend().remove()
# plt.xscale('log')
ax.tick_params(axis='y', which='major', length=0)
# ax.get_xaxis().set_visible(False)

for tick in ax.get_xticklabels():
    tick.set_fontname("Helvetica")
for tick in ax.get_yticklabels():
    tick.set_fontname("Helvetica")

df = df.reset_index()  # Resetting index for the below annotations to work
for i, r in df.iterrows():
    s = str("%.1f" % r["hydrophob diff"])
    if r["hydrophob diff"] < 0:
        ax.text(0, i-0.4,  s, fontname="Helvetica")
    else:
        ax.text(r["hydrophob diff"], i-0.4,  s, fontname="Helvetica")

ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_xlabel("", fontname="Helvetica")
ax.xaxis.labelpad = -2

ax.set_title("hydrophobicity\ndifference", fontname="Helvetica")

In [ ]:
# For display in slides
df = aa_pos_prop_df[[#'case',
                     'size', 'chain type', 'polar', 'common in',
                     'hydrophobic',
                     'iron binding',
                     'carbonylation',
                     'oxygen rich',
                     'photooxidation',
                     'pathogen',
                     'order',
                     'TM stabilizing',]]
df = df.reindex(['S', 'W', 'Y', 'F', 'I','V', 'L','N', 'D', 'E', 'C', 'P', 'R', 'M', 'G', 'H', 'A', 'K', 'T', 'Q'])
df

In [ ]:
# For use with UpSetR
df = aa_pos_prop_df[[#'case',
                     'size', 'chain type', 'polar', 'common in',
                     'hydrophobic',
                     'iron binding',
                     'carbonylation',
                     'oxygen rich',
                     'photooxidation',
                     'pathogen',
                     'order',
                     'TM stabilizing',]]
df